##### MY470 Computer Programming

### Problem Set 3, AT 2023

#### \*\*\* Due 12:00 noon on Monday, November 6 \*\*\*

---
### Simulating contagion on a network

In this problem set, you are asked to write a program that simulates the contagion of disease or information on an empirical network. In academic research, contagion models have been used to study the properties of different types of networks. In practice, contagion models are extremely valuable to predict the spread of contagious disease such as the flu or STDs.

We will use the simplest of contagion models — the SI model. SI stands for susceptible and infected. The SI model assumes that once a susceptible individual is infected, there is no recovery. This is a good representation for the spread of non-curable but non-deadly infectious disease such as Herpes simplex or for the spread of new technologies and knowledge.

In the SI model we will implement, we will start with a population where everyone is susceptible. Then we will randomly pick a small number of individuals ("Patients 0") and infect them. In the next period, all the contacts of the infected individuals will get infected (thus, we will assume that the probability of transmission is 1). And so on. We will repeat the process until everyone in the network is infected or until a certain number of periods have passed (the latter is necessary for networks that are not connected and have separate components; in such situations, it is possible that the contagion never reaches some individuals). 

We will run the model on a real network. For simplicity, we will reuse the co-authorship network we analyzed in Problem Set 1. So think about the contagion in this case as learning about a new research technique, empirical finding, or theoretical result.

#### Hints

Use docstrings to describe your methods. We will subtract points from your mark if you do not use appropriate description of your code.


### Problem 1: Working in a team

Work with your assigned partner to complete and submit the problem set. You can meet in person to discuss the division of labor but we expect you to use GitHub to communicate when coding your part and merging your contributions. We will  review the Issues, Pull request, and Wiki stats for your repository. You will get the full points for this problem if we find sufficient evidence that you have made use of GitHub as a collaboration tool. 

#### Hints

One reasonable way to divide the work is to assign Problems 2 and 3 to Student A and Problems 4 and 5 to Student B.


### Problem 2: Class for network

Create a class called `UndirectedNetwork`. The class should have the following data attributes:

* `nodes` — a dictionary where the node id is a key and the value is a list with the ids of the node's neighbors (coauthors for our data); initially empty

and the following methods:

* `add_node` — takes `node_id` and initializes it as a key to `nodes` if it is not already there
* `add_neighbors` — takes two arguments: `ego_id` and `alter_id` and adds `alter_id` to `ego_id`'s list of neighbors and `ego_id` to `alter_id`'s list of neighbors, if they are not already there
* `get_node_ids` — generator method that gives the ids of the nodes in the network
* `get_node_neighbors` — generator method that takes `node_id` and gives its neighbors

Calling the `print()` function on a `UndirectedNetwork` object should print the number of nodes in the network, e.g. "Undirected network with 455 nodes".


In [1]:
# Enter your answer to Problem 2 below.
class UndirectedNetwork:
    """A class representing an undirected network."""
    def __init__(self):
        """Initialize an empty UndirectedNetwork."""
        self.nodes = {}
    
    def add_node(self, node_id):
        """Add a node to the network if it doesn't already exist."""
        if node_id not in self.nodes:
            self.nodes[node_id] = []
    
    def add_neighbors(self, ego_id, alter_id):
        """Add an undirected edge between two nodes."""
        if alter_id not in self.nodes[ego_id]:
            self.nodes[ego_id].append(alter_id)
        if ego_id not in self.nodes[alter_id]:
            self.nodes[alter_id].append(ego_id)
    
    def get_node_ids(self):
        """Iterate over all node IDs in the network.
        This generator function yields the unique identifiers of all nodes in the network.
        """
        for node_id in self.nodes:
            yield node_id
    
    def get_node_neighbors(self, node_id):
        """Iterate over the neighbors of a specific node in the network.
        This generator function yields the unique identifiers of neighbors for a given node in the network.
        """
        if node_id in self.nodes:
            for neighbor_id in self.nodes[node_id]:
                yield neighbor_id
    
    def __str__(self):
        """Get a string representation of the network,  indicating the number of nodes it contains."""
        return f"Undirected network with {len(self.nodes)} nodes"

### Problem 3: Create an instance of the network class

Read the data from the file `ca-GrQc.txt` in the `data` repository (use the same relative path as in the previous problem sets). Save the data in an instance of the UndirectedNetwork class you created. Call print on the instance.


In [2]:
# Enter your answer to Problem 3 below.
network = UndirectedNetwork()
    
with open('../data/ca-GrQc.txt', 'r') as file:
    for line in file:
        if line.startswith('#'):
            continue
        ego_id, alter_id = map(int, line.strip().split())
        network.add_node(ego_id)
        network.add_node(alter_id)
        network.add_neighbors(ego_id, alter_id)
   
print(network)

Undirected network with 5242 nodes


---
### Problem 4: Class for SI model


Create a class called `SIModel` that has the following data attributes:

* `network` — an instance of class UndirectedNetwork taken at instantiation
* `susceptible_nodes` — a list of ids for nodes that are not yet infected; initially includes all nodes from `network`
* `infected_nodes` — a list of ids for nodes that are infected; initially empty
* `num_infected` — keeps track of the number of infected nodes; initially `0`

and the following methods:

* `initialize` — takes an integer `n` to randomly select `n` number of nodes and infect them; then prints the number of infected nodes
* `update` — iterates over the susceptible nodes in random order and infects those who have at least one infected neighbor; then prints the number of infected nodes. The process should be asynchronous, in the sense that a node immediately becomes infected and will then infect any susceptible neighbors who are yet to be iterated over.
* `run` — repeats `update` until all nodes are infected or until `update` has been run 30 times

Calling the `print()` function on a `SIModel` object should print `num_infected`.

#### Hints

In this problem you will need to use functions from the `random` module. You can read more about it [here](https://docs.python.org/3/library/random.html).

Make sure the methods update all the relevant data attributes when called.

In [3]:
# Enter your answer to Problem 4 below. 
import random

class SIModel(object):  

    def __init__(self, network):
        """Constructor method that initializes the SIModel class, taking as an argument an instance 
        of the UndirectedNetwork class provided during instantiation."""
        self.network = network 
        self.susceptible_nodes = list(self.network.get_node_ids())  
        self.infected_nodes = []
        self.num_infected = 0

    def initialize(self, n):
        """Initializes the SIModel by randomly selecting and infecting 'n' nodes from the network."""
        #We establish this condition to not get an error, in case the random n is higher than our actual population
        if n  > len(self.susceptible_nodes):
            n = len(self.susceptible_nodes)

        #We infect randomly selected n number of nodes
        infectednodes = random.sample(self.susceptible_nodes, n)

        #We add those to the list of infected nodes 
        self.infected_nodes.extend(infectednodes)

        #We remove from the susceptible nodes the ones that have been infected now
        for infectednode in infectednodes:
            self.susceptible_nodes.remove(infectednode)

        #We add the randomly infected nodes to the total number of infected
        self.num_infected += n    
        print(self.num_infected)


    def update(self):
        """Iterates over susceptible nodes in a random order and infects those having at least one infected neighbor."""
        for node in random.sample(self.susceptible_nodes, len(self.susceptible_nodes)):
            neighbors = list(self.network.get_node_neighbors(node)) 
            if any(neighbor in self.infected_nodes for neighbor in neighbors):
                self.infected_nodes.append(node)
                self.susceptible_nodes.remove(node)

        #We update all the varibales as in the previous step       
        self.num_infected = len(self.infected_nodes)   
        print(self.num_infected)


    def run(self):
        """Repeatedly runs the update method until all nodes are infected or until the method has been executed 30 times."""
        iteration = 0

        while iteration < 30 and len(self.susceptible_nodes) > 0:
            iteration += 1
            self.update()


    def __str__(self):
        """Returns a string representation of the SIModel instance."""
        return str(self.num_infected) 

---
### Problem 5: Run the model

Run `SIModel` using the network from Problem 2. You should initialize the simulation with 3 seeds (the number of "patients 0").


In [4]:
# Enter your answer to Problem 5 below. 
si_model = SIModel(network)
si_model.initialize(3)
si_model.run()

3
824
3069
3979
4126
4152
4156
4158
4158
4158
4158
4158
4158
4158
4158
4158
4158
4158
4158
4158
4158
4158
4158


---

### Evaluation

| Problem | Mark     | Comment   
|:-------:|:--------:|:----------------------
| 1       |   1/2    |  Could make a little more use raising, resolving, closing issues .
| 2       |   4/4    |  You could also check that ego/alter are present in the network (and add them) within `add_neighbors`   
| 3       |   1/1    | You should check that you don't add any self-loops to the network
| 4       |   5/5    | Good. Can just do `for neighbor in self.network.get_node_neighbors(node)`, rather than creating list.
| 5       |   1/1    | Good
| Legibility   |   1/2    | docstring needed for SIModel
| Modularity   |   1/2    | Can just use `print(self)` for num infected
| Efficiency   |   3/3    | Good
|**Total**|**17/20**  | Very good
